## IMDb 영화 리뷰 데이터 셋

In [ ]:
import tensorflow as tf
from tensorflow import keras

IMDb 영화 리뷰 데이터셋 적재

In [ ]:
(X_train, y_train), (X_test, y_test)=keras.datasets.imdb.load_data()
X_train[0][:10]

17473536/17464789 [==============================] - 0s 0us/step


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

구두점을 모두 제거하고 소문자로 변환한 후, 공백으로 나누어 빈도에 따라 인덱스로 붙여 넘퍼이 정수 배열로 표현되어 있는 것을 볼 수 있습니다.

이때 0, 1, 2는 일반적인 단어가 아닌 패딩 토큰, SOS 토큰, 알 수 없는 단어를 의미합니다.

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
  id_to_word[id_] = token

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

IMDb 데이터셋에서는 이미 전처리가 되어 있지만, 실제 프로젝트에서는 직접 텍스트 전처리를 진행해야 합니다!

전처리 과정에서 구두점, 줄바꿈, 탭을 포함한 많은 글자들이 제외되는데, 공백 제거를 통해 전처리를 진행할 수 있습니다. 하지만 언어마다 공백 사용법도 다르고 공백에 따라 단어 의미(San Francisco)가 달라질 수도 있기 때문에 이 방법은 최선이 아닙니다.


다양한 전처리 방법
*   Taku Kudo - 부분 단어 수준의 토큰화 및 복원의 비지도 학습 방법
*   Rico Sennrich - 부분 단어의 인코딩
*   TF.Text - https://www.tensorflow.org/text




텐서플로 연산만을 사용한 데이터 전처리 과정에 대해 알아봅시다!

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteELITA0/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteELITA0/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
def preprocess(X_batch, y_batch):
    # 리뷰의 300글자
    X_batch = tf.strings.substr(X_batch, 0, 300)
    # 정규식을 이용해 "<br/ >"" -> " "
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")

    # 정규식을 이용해 a~Z와 작은 따옴표를 제외한 모든 문자 -> " "
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")

    # 공백 기준으로 파싱
    X_batch = tf.strings.split(X_batch)

    # 래그드 텐서(리스트의 리스트)를 밀집 텐서로 변환
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [ ]:
from collections import Counter

# 단어의 등장 횟수 계산
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [ ]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

가장 많이 나온 단어 10000개만 남기고 삭제합니다.



In [ ]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

10000개의 단어를 ID(인덱싱) 단계를 진행합니다. OOV 버킷을 사용하는 룩업 테이블을 만듭니다.

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

'faaaaaantastic'은 lookup 테이블에 존재하지 않으므로 10000보다 큰 ID를 가지게 됩니다.

Batch로 묶은 후 전처리를 통해 짧은 시퀀스로 바꿔줍니다. 위에서 만든 룩업 테이블을 이용해 단어를 인코딩합니다.

In [ ]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 49s 47ms/step - loss: 0.5417 - accuracy: 0.7149
Epoch 2/5
782/782 [==============================] - 35s 45ms/step - loss: 0.3487 - accuracy: 0.8549
Epoch 3/5
782/782 [==============================] - 35s 45ms/step - loss: 0.1930 - accuracy: 0.9298
Epoch 4/5
782/782 [==============================] - 35s 45ms/step - loss: 0.1374 - accuracy: 0.9506
Epoch 5/5
782/782 [==============================] - 36s 46ms/step - loss: 0.1144 - accuracy: 0.9574


## 마스킹

마스킹 작업: 차원 내 원소의 위치를 표시하는 작업  
왜 필요할까요?  
Ex)  I like Tave.  
기대되는 입력층 : [~ ,~ ,~ ,~ ,~ ,~]  
패딩이 포함된 입력층 : ['I', 'like', 'Tave', '0', '0', '0'] => [12 ,51 ,100002 , \<pad> , \<pad> , \<pad>]  
이때 패딩은 의미가 없는 정보가 되므로 앞 3 단어가 의미있는 단어라는 것을 알려줘야합니다!

마스크 텐서([True, True, True, False, False, False])가 생성되어 모든 층에 타임 스텝 차원이 유지되는 동안 자동으로 전파되어야 합니다.

In [ ]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])

mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 44s 45ms/step - loss: 0.5456 - accuracy: 0.7180
Epoch 2/5
782/782 [==============================] - 35s 45ms/step - loss: 0.3619 - accuracy: 0.8489
Epoch 3/5
782/782 [==============================] - 35s 45ms/step - loss: 0.1978 - accuracy: 0.9275
Epoch 4/5
782/782 [==============================] - 35s 45ms/step - loss: 0.1380 - accuracy: 0.9512
Epoch 5/5
782/782 [==============================] - 35s 45ms/step - loss: 0.1080 - accuracy: 0.9606


성능이 굉장히 좋게 나왔습니다!  
더 많은 리뷰로 학습을 한다면 좋은 임베딩을 만들 수 있지만 현실적으로는 데이터가 부족하여 힘듭니다. 

## 사전훈련된 임베딩 재사용하기

텐서플로 허브를 이용해 사전 훈련된 모델 컴포넌트를 쉽게 추가할 수 있습니다.

In [ ]:
import tensorflow_hub as hub

model = keras.Sequential([
    # 문장 인코더(모듈) 다운로드
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    # 감성분석 모델
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

문장 인코더는 문자열을 입력받아 하나의 벡터로 인코딩합니다.  내부적으로는 문자열 파싱 후 대규모 코퍼스로 사전 훈련된 임베딩 행렬을 사용해 각 단어를 임베딩합니다. 그 후 모든 단어 임베딩의 평균을 계산합니다.

문장 임베딩 후, 두 개의 Dense층을 추가해 감성 분석 모델을 구성합니다.

이렇게 모델을 구성한 후, 배치와 프리패치를 진행하고 모델을 훈련하면됩니다.

In [ ]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32 

Epoch 1/5
782/782 [==============================] - 8s 9ms/step - loss: 0.5444 - accuracy: 0.7295
Epoch 2/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5145 - accuracy: 0.7484
Epoch 3/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5094 - accuracy: 0.7506
Epoch 4/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5058 - accuracy: 0.7538
Epoch 5/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5026 - accuracy: 0.7556


기본적으로 TF허브는 다운로드한 파일을 로컬 시스템의 임시 디렉터리에 저장합니다. 다음 코드로 고정 디렉터리에 저장할 수 있습니다.

In [ ]:
import os
os.environ['TFHUB_CACHE_DIR']="./my_tfhub_cache"